# Transformer 구현하기 2
Encoder, Decoder 및 Transformer 모델 전체에 대한 설명


## 1. Config
Transformer 모델에서는 많은 설정이 필요하다. 이런 설정들을 json 형태로 저장하고 이를 읽어서 처리하는 클래스

In [ ]:
""" configuration json을 읽어들이는 class """
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)


작은 리소스에서도 동작 가능하도록 여러 파라미터를 작게 설정. GPU가 여유가 있다면 파라미터를 키우면 더 좋은 결과를 확인 할 수 있을 겁니다. 기본 파라미터는 config.json을 참고

In [ ]:
config = Config({
    "n_enc_vocab": len(vocab),
    "n_dec_vocab": len(vocab),
    "n_enc_seq": 256,
    "n_dec_seq": 256,
    "n_layer": 6,
    "d_hidn": 256,
    "i_pad": 0,
    "d_ff": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12
})
print(config)

**위로 받은 파라미터 정보**  
{'n_enc_vocab': 8007, 'n_dec_vocab': 8007, 'n_enc_seq': 256, 'n_dec_seq': 256, 'n_layer': 6, 'd_hidn': 256, 'i_pad': 0, 'd_ff': 1024, 'n_head': 4, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12}


## 2. Common Class
### Position Encoding
![](https://paul-hyun.github.io/assets/2019-12-19/sinusoid_encoding_table.png)
Position Embedding의 초기 값을 구하는 함수. 
    1. 각 position별 hidden index 별, angle 값을 구한다.
    2. hidden 짝수 index의 angle 값의 sin값을 취한다.
    3. hedden 홀수 index의 angle 값의 cos값을 구한다.

    

In [ ]:
""" sinusoid position encoding """
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos

    return sinusoid_table


### Attention Pad Mask
![](https://paul-hyun.github.io/assets/2019-12-19/pad_mask.png)
attention을 구할때, Padding 부분을 제외하기 위한 마스크를 구하는 함수
    1. K의 값중 Pad인 부분을 True로 변경. 나머지는 False
    2. 구해진 값의 크기를 Q-len, K-len 되도록 변경

In [ ]:
""" attention pad mask """
def get_attn_pad_mask(seq_q, seq_k, i_pad):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(i_pad)
    pad_attn_mask= pad_attn_mask.unsqueeze(1).expand(batch_size, len_q, len_k)
    return pad_attn_mask


### Attention Decoder Mask
![](https://paul-hyun.github.io/assets/2019-12-19/decoder_mask.png)
Decoder의 Masked Multi Head Attention에서 사용할 Mask를 구하는 함수. 현재 단어와 이전 단어는 볼수 있고, 다음 단어는 볼수 없도록 마스킹
    1. 모든 값이 1인 Q-len, K-len 테이블을 생성
    2. 대각선 기준으로 아래쪽을 0으로 만든다.

In [ ]:
""" attention decoder mask """
def get_attn_decoder_mask(seq):
    subsequent_mask = torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0), seq.size(1), seq.size(1))
    subsequent_mask = subsequent_mask.triu(diagonal=1) # upper triangular part of a matrix(2-D)
    return subsequent_mask


### Scaled Dot Product Attention
![](https://paul-hyun.github.io/assets/2019-12-19/scale_dot_product_attention.png)

    1. Q * K.Transpose를 구한다.
    2. K-dimension에 루트를 취한 값으로 나눠 준다.
    3. Mask를 적용.
    4. Softmax를 취해 각 단어의 가중치 확률 분포 attn_prob를 구한다.
    5. attn_prob * V를 구한다. 구한값은 Q에 대한 V의 가중치 합 벡터

In [ ]:
""" scale dot product attention """
class ScaledDotProductAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.dropout = nn.Dropout(config.dropout)
        self.scale = 1 / (self.config.d_head ** 0.5)
    
    def forward(self, Q, K, V, attn_mask):
        # (bs, n_head, n_q_seq, n_k_seq)
        scores = torch.matmul(Q, K.transpose(-1, -2))
        scores = scores.mul_(self.scale)
        scores.masked_fill_(attn_mask, -1e9)
        # (bs, n_head, n_q_seq, n_k_seq)
        attn_prob = nn.Softmax(dim=-1)(scores)
        attn_prob = self.dropout(attn_prob)
        # (bs, n_head, n_q_seq, d_v)
        context = torch.matmul(attn_prob, V)
        # (bs, n_head, n_q_seq, d_v), (bs, n_head, n_q_seq, n_v_seq)
        return context, attn_prob

### Multi-Head Attention
![](https://paul-hyun.github.io/assets/2019-12-19/multi_head_attention.png)
Multi-Head Attention을 구하는 클래스
    1. Q * w_Q를 한 후 multi-head로 나눈다.
    2. K * w_K를 한 후 multi-head로 나눈다.
    3. V * w_V를 한 후 multi-head로 나눈다.
    4. ScaledDotProductAttention 클래스를 이용해, 나누어진 head 별로 Attention을 구한다.
    5. 여러개의 head를 다시 1개로 합친다.
    6. 합친 head에 대해 Linear를 취해, 최종 Multi-Head Attention 값을 구한다.

In [ ]:
""" multi head attention """
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.W_Q = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_K = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_V = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.scaled_dot_attn = ScaledDotProductAttention(self.config)
        self.linear = nn.Linear(self.config.n_head * self.config.d_head, self.config.d_hidn)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        # (bs, n_head, n_q_seq, d_head)
        q_s = self.W_Q(Q).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        # (bs, n_head, n_k_seq, d_head)
        k_s = self.W_K(K).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        # (bs, n_head, n_v_seq, d_head)
        v_s = self.W_V(V).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)

        # (bs, n_head, n_q_seq, n_k_seq)
        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.config.n_head, 1, 1)

        # (bs, n_head, n_q_seq, d_head), (bs, n_head, n_q_seq, n_k_seq)
        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        # (bs, n_head, n_q_seq, h_head * d_head)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.config.n_head * self.config.d_head)
        # (bs, n_head, n_q_seq, e_embd)
        output = self.linear(context)
        output = self.dropout(output)
        # (bs, n_q_seq, d_hidn), (bs, n_head, n_q_seq, n_k_seq)
        return output, attn_prob

### Feed Forward
![](https://paul-hyun.github.io/assets/2019-12-19/feed-forward.png)
 Feed Forward를 처리한다.
 
     1. Linear를 실행하여 shape을 d_ff(hidden * 4) 크기로 키운다.
     2. activation 함수(relu, gelu)를 실행한다.
     3. Linear를 실행하여, shape을 hidden 크기로 줄여준다.

In [ ]:
""" feed forward """
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.conv1 = nn.Conv1d(in_channels=self.config.d_hidn, out_channels=self.config.d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config.d_ff, out_channels=self.config.d_hidn, kernel_size=1)
        self.active = F.gelu
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, inputs):
        # (bs, d_ff, n_seq)
        output = self.conv1(inputs.transpose(1, 2))
        output = self.active(output)
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        output = self.dropout(output)
        # (bs, n_seq, d_hidn)
        return output

## 3. Encoder
![](https://paul-hyun.github.io/assets/2019-12-19/encoder.png)

### Encoder Layer

Encoder에서 루프를 돌며 처리할 수 있도록 EncoderLayer를 정의하고, 여러개 만들어서 실행한다.  
    1. Multi-Head Attention을 수행. Q, K, V 모두 동일한 값을 사용하는 self-attention
    2. 1번의 결과와 input(residual)을 더한 후 LayerNorm을 실행한다.
    3. 2번의 결과를 입력으로 Feed Forward를 실행한다.
    4. 3번의 결과와 2번의 결과(residual)을 더한 후 LayerNorm을 실행
    

In [ ]:
""" encoder layer """
class EncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, inputs, attn_mask):
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        att_outputs, attn_prob = self.self_attn(inputs, inputs, inputs, attn_mask)
        att_outputs = self.layer_norm1(inputs + att_outputs)
        # (bs, n_enc_seq, d_hidn)
        ffn_outputs = self.pos_ffn(att_outputs)
        ffn_outputs = self.layer_norm2(ffn_outputs + att_outputs)
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        return ffn_outputs, attn_prob


### Encoder
Encoder 클래스.  
    1. 입력에 대한 Position 값을 구한다.
    2. Input Embedding과 Position Embedding을 구한 후 합한다.
    3. 입력에 대한 attention_pad_mask를 구한다.
    4. for 루프를 돌며, 각 layer를 실행한다.
> Layer의 입력은 이전 layer의 출력값

In [ ]:
""" encoder """
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.enc_emb = nn.Embedding(self.config.n_enc_vocab, self.config.d_hidn)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_enc_seq + 1, self.config.d_hidn))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)

        self.layers = nn.ModuleList([EncoderLayer(self.config) for _ in range(self.config.n_layer)])
    
    def forward(self, inputs):
        positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
        pos_mask = inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)

        # (bs, n_enc_seq, d_hidn)
        outputs = self.enc_emb(inputs) + self.pos_emb(positions)

        # (bs, n_enc_seq, n_enc_seq)
        attn_mask = get_attn_pad_mask(inputs, inputs, self.config.i_pad)

        attn_probs = []
        for layer in self.layers:
            # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
            outputs, attn_prob = layer(outputs, attn_mask)
            attn_probs.append(attn_prob)
        # (bs, n_enc_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return outputs, attn_probs

## 4. Decoder
![](https://paul-hyun.github.io/assets/2019-12-19/decoder.png)
decoder에서 루프를 돌면서 처리 할 수 있도록 DecoderLayer를 정의하고, 여러개를 만들어서 실행한다.
    1. Multi-Head Attention을 수행.
    Q, K, V 모두 동일한 값을 사용하는 self-attention 사용
    2. 1번의 결과와 input을 더한(residual) 후 LayerNorm을 실행
    3. Encoder-Decoder Multi-HEad Attention을 수행
        Q: 2번의 결과
        K, V: Encoder 결과
    4. 3번의 결과와 2번의 결과를 더한 후 LayerNorm을 실행
    5. 4번의 결과를 입력으로 Feed Forward 실행.
    6. 5번의 결과와 4번의 결과(residual)을 더한 후 LayerNorm을 실행 

In [ ]:
""" decoder layer """
class DecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.dec_enc_attn = MultiHeadAttention(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm3 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, dec_inputs, enc_outputs, self_attn_mask, dec_enc_attn_mask):
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq)
        self_att_outputs, self_attn_prob = self.self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask)
        self_att_outputs = self.layer_norm1(dec_inputs + self_att_outputs)
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_enc_seq)
        dec_enc_att_outputs, dec_enc_attn_prob = self.dec_enc_attn(self_att_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_enc_att_outputs = self.layer_norm2(self_att_outputs + dec_enc_att_outputs)
        # (bs, n_dec_seq, d_hidn)
        ffn_outputs = self.pos_ffn(dec_enc_att_outputs)
        ffn_outputs = self.layer_norm3(dec_enc_att_outputs + ffn_outputs)
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq), (bs, n_head, n_dec_seq, n_enc_seq)
        return ffn_outputs, self_attn_prob, dec_enc_attn_prob